## News :D



In [27]:
%load_ext autoreload
%autoreload 2

import requests
import re 
import itertools
from bs4 import BeautifulSoup # Imports bs4
import wget # Imports wget
import articles
import sitemaps
import test_functions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

### Data formats

CNN has links by month

NYT has links by day

BI has links by month

TH has links by day

NYP has links by day

DM has links by day

Fox has links by page num

### Thoughts on how to organize the data 

we use a pandas dataframe organized like this

| Dates | CNN | NYT | BI | TH | NYP | DM | FOX |
| ----------- | ------------------------------------------ | ------- | ------- | ------- | ------- | ------- | ------- |
| 2014-01-01  | list of tuples with article link & title   | ... | ... | ... | ... | ... | ... |
| 2014-01-02  | ... | ... | ... | ... | ... | ... | ... |
| 2014-01-03  | ... | ... | ... | ... | ... | ... | ... |
| 2014-01-04  | ... | ... | ... | ... | ... | ... | ... |
| ...         | ... | ... | ... | ... | ... | ... | ... |
| 2024-12-31  | ... | ... | ... | ... | ... | ... | ... |

### Code to get data
CNN:
```html
<li>
    <span class="date">2024-01-31</span>
    <span class="sitemap-link">
        <a href="https://www.cnn.com/tech/live-news/meta-x-discord-tiktok-snap-chiefs-testimony-senate/index.html">
            Mark Zuckerberg apologizes to families over social media harms in contentious Senate hearing
        </a>
    </span>
</li>
```

NYT:
```html
<li>
    <a href="https://www.nytimes.com/2023/12/31/business/dealbook/pga-tour-saudi-deal-deadline.html">
        PGA Tour and Saudi-Backed LIV Extend Deadline to Finalize Deal
    </a>
</li>
```

BI:
```html
<p>
    <a href="https://www.businessinsider.com/feud-capote-vs-swans-ann-woodward-suicide-death-real-story-2024-1">
        The real story behind the suicide of American socialite Ann Woodward in &#39;Feud: Capote vs. The Swans&#39;
    </a>
    <br>2024-02-01T03:31:01.68Z
</p>
```

NYP:
```html
<h3 class="story__headline headline headline--archive">
    <a href="https://nypost.com/2024/01/01/sports/og-anunoby-trending-toward-the-perfect-piece-for-knicks/" rel="" target="_self">
		Knicks&#039; big trade acquisition already looks like the &#039;perfect fit&#039; 												
    </a>
</h3>
```

DM:
```html
<a href="/wires/ap/article-12917573/Christian-McCaffrey-miss-49ers-regular-season-finale-return-playoffs.html" title="">
    Christian McCaffrey will miss the 49ers regular-season finale but should return for playoffs
</a>
```

FOX:
```html
<loc>
    https://www.foxnews.com/opinion/why-the-department-of-education-is-going-to-rip
</loc>
<lastmod>
    2025-03-12T05:00:54-04:00
</lastmod>
```

In [3]:
[cnn_dict, nyt_dict, bi_dict, nyp_dict, dm_dict, fox_links] = sitemaps.sitemaps()

In [23]:
start_year = 2020
end_year = 2020
start_month = 1
end_month = 1
news = [1,1,1,0,0,1] # determines which news sources we scrape (for testing)       0 = don't   1 = do       CNN, NYT, BI, NYP, DM, FOX

[cnn_articles, nyt_articles, bi_articles, nyp_articles, dm_articles, fox_articles] = articles.articles(news,start_year,end_year,start_month,end_month)

/home/grant/Documents/HeadlineCorrelation/articles.py:374: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(r.text, 'lxml')


In [54]:
# Pandas Testing area
import pandas as pd
import numpy as np

mday = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

news_names = ["CNN", "NYT", "BI", "NYP", "DM", "FOX"]
news_data = {}
example_data = {}

for year in range(start_year,end_year+1):
    for month in range(start_month,end_month+1):
        for day in range(mday[month-1]):
            news_data[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"] = [
                cnn_articles[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"],
                nyt_articles[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"],
                bi_articles[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"],
                'N/A',
                'N/A',
                fox_articles[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"]
            ]
            example_data[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"] = [
                'list',
                'list',
                'list',
                'list',
                'list',
                'list'
            ]

test_frame = pd.DataFrame(news_data, index=news_names)
example_test_frame = pd.DataFrame(example_data, index=news_names)

# print(test_frame)
# print(example_test_frame)

print(test_frame['2020-01-01'].loc["CNN"])

[('Princeton University is hanging a series of portraits that honor its blue-collar campus workers', 'https://www.cnn.com/2020/01/01/us/princeton-university-portraits-workers-trnd/index.html'), ('Nanobots, ape chauffeurs and flights to Pluto. The predictions for 2020 we got horribly wrong', 'https://www.cnn.com/2020/01/01/tech/2020-predictions-we-got-wrong-scli-intl/index.html'), ('A man working at Starbucks had his laptop stolen. He was killed when he chased down the thief', 'https://www.cnn.com/2020/01/01/us/oakland-laptop-thief-starbucks-trnd/index.html'), ("Magic Johnson says David Stern's response to HIV announcement changed the world", 'https://www.cnn.com/2020/01/01/us/david-stern-death-reactions/index.html'), ('A transgender character is coming to the Marvel Universe', 'https://www.cnn.com/2020/01/01/entertainment/transgender-marvel-character-trnd/index.html'), ('National Highway Traffic Safety Administration is investigating a Tesla crash that killed 2 people', 'https://www.cn

In [92]:
test_news = [1,0,0,0,0,1]

[words, title_counts] = test_functions.most_common(test_news,start_year,end_year,start_month,end_month,'Trump',test_frame)

for source in words:
    del_words = []

    for i in words[source]:
        words[source][i] = (words[source][i] / title_counts[source]) * 100

    for i in words[source]:
        if words[source][i] <= 1:
            del_words.append(i)
    for i in del_words:
        del words[source][i]


dels = []
for i in words['CNN']:
    if i not in words['FOX']:
        dels.append(i)
for i in dels:
    del words['CNN'][i]

dels = []
for i in words['FOX']:
    if i not in words['CNN']:
        dels.append(i)
for i in dels:
    del words['FOX'][i]

for word in words['CNN']:
    print(word)
    print(f"CNN: {words['CNN'][word]}")
    print(f"FOX: {words['FOX'][word]}")
    print("")


killing
CNN: 1.1764705882352942
FOX: 1.728395061728395

campaign
CNN: 1.411764705882353
FOX: 1.9753086419753085

vote
CNN: 1.411764705882353
FOX: 1.8518518518518516

witnesses
CNN: 1.411764705882353
FOX: 2.4691358024691357

it
CNN: 1.647058823529412
FOX: 1.2345679012345678

out
CNN: 1.647058823529412
FOX: 1.4814814814814816

what
CNN: 1.647058823529412
FOX: 1.6049382716049383

him
CNN: 1.647058823529412
FOX: 1.3580246913580247

will
CNN: 1.8823529411764703
FOX: 1.728395061728395

deal
CNN: 1.8823529411764703
FOX: 1.4814814814814816

strike
CNN: 2.1176470588235294
FOX: 1.728395061728395

new
CNN: 2.1176470588235294
FOX: 2.345679012345679

plan
CNN: 2.3529411764705883
FOX: 1.4814814814814816

has
CNN: 2.588235294117647
FOX: 1.9753086419753085

war
CNN: 2.588235294117647
FOX: 3.2098765432098766

over
CNN: 2.588235294117647
FOX: 2.345679012345679

trade
CNN: 2.588235294117647
FOX: 1.1111111111111112

he
CNN: 2.823529411764706
FOX: 2.345679012345679

legal
CNN: 2.823529411764706
FOX: 2.4691